# <center>Informatique tc3 (Projet Web) - TD1</center>

## <center style="color: #66d">Introduction au Ray Tracing</center>


## 0. Remarques préliminaires

### 0.1 Présentation du cycle de TD

Le travail qui vous sera proposé dans le cadre de votre projet consiste à développer un serveur de synthèse d'images basées sur le principe du lancer de rayons, accessible et pilotable via une interface Web. Le lancer de rayon, ou _ray tracing_ en anglais, est une technique de synthèse d'images réalistes par ordinateur.

Dans le cadre de ce premier TD, vous aurez l'occasion de découvrir le principe de la synthèse d'images par lancer de rayons. Le déroulement du TD dévoile pas à pas les mécanismes mis en oeuvre pour obtenir des images réalistes. Le but n'est pas de finir absolument le sujet en 2h, mais que chacun avance à son rythme et comprenne les raisonnements et les solutions mises en jeu. L'objectif pédagogique est de vous fournir une base de travail pour la compréhension des principes du ray-tracing sur lesquels vous pourrez revenir et progresser tout au long de l'avancement de votre projet.

Lors du second TD nous étudierons comment utiliser une base de données pour stocker et réutiliser divers types d'information.
Nous verrons en particulier comment sauvegarder les paramètres ayant permis de créer une image donnée &ndash; position et couleurs des objets, etc.

Le troisième TD sera consacré à la mise en place d'un serveur web, que nous munirons d'un cache pour éviter de regénérer plusieurs fois des images identiques, coûteuses en temps de calcul.

La mise en place d'une interface Web permettant les interactions avec l'application développée par chacun des groupes de projet fera l'objet du TD 4.

__N.B.__ Il est plus que probable qu'il ne vous soit pas possible de finir chacun des TDs en 2h. Toutefois, les sujets sont conçus pour vous faire avancer dans votre projet.
Il est donc fortement recommandé d'aller le plus loin possible dans le sujet proposé pour chacun des TDs, __mais jamais au détriment du TD suivant__. Autrement dit : ne sacrifiez pas la séance du TD2 pour finir le TD1. Cette remarque vaut bien sûr pour l'ensemble de la série.


### 0.2 Choix du sujet de projet

Suite à la demande des générations précédentes d'étudiants, la constitution des groupes et le choix du sujet seront effectués lors de cette première séance, en amont des TDs. Les sujets sont les suivants :
<ul>
<li style="margin-top:1em"> 1. Visualisation de molécules. Interface permettant le choix de la molécule, sa position, son orientation, le facteur de zoom, l'éclairage, le fond. Pour la description de molécules on pourra se baser sur des molécules existantes, décrites au format <tt>.mol</tt> (extension de fichier) que l'on devra analyser et importer.

<li style="margin-top:1em"> 2. Gestion des sources d'éclairage. Création de nouveaux types de sources (située à l'infini, conique...). Sur la base d'une scène unique ou d'un jeu de scènes prédéfinies, interface permettant l'ajout et la suppression de sources, et la modification des caractéristiques des sources (nature, position, orientation, couleur).

<li style="margin-top:1em"> 3. &Eacute;diteur de scène. &Agrave; partir de l'exemple du cube et du cylindre, création d'une collection d'objets composés (pyramide, maison, arbre...). Interface permettant la création de scènes basées sur des objets composés.

<li style="margin-top:1em"> 4. Gestion de la caméra. Amélioration du code permettant de modifier la position de la caméra. Interface permettant de jouer avec la caméra sur la base d'une scène donnée ou d'un jeu de scènes prédéfinies.

<li style="margin-top:1em"> 5. Création de séquences vidéo animées. Interface permettant le choix des caractéristiques techniques de l'animation (nombre d'images, nombre d'images par seconde, taille des images) ainsi que du (ou des)  paramètre(s) variant au cours de l'animation (déplacement d'objets, de l'éclairage, de la caméra...) sur la base d'une scène donnée ou d'un jeu de scènes prédéfinies.
</ul>

<span style="color:red">__N.B.__ Les sujets de projet vous seront détaillés  en temps utile...</span>

### 0.3 Déroulement du TD

Si vous lisez ces lignes depuis votre poste de travail, vous avez &ndash; c'est la bonne méthode :
    <ul style="margin-top: 0">
    <li> créé un répertoire sur votre machine pour les fichiers relatifs à ce TD,
    <li> récupéré le sujet <i>(fichier zip)</i> sur le serveur pédagogique,
    <li> dézippé le tout et placé le contenu du fichier zip dans le répertoire que vous venez de créer,
    <li> démarré un serveur de notebook pour consulter le présent sujet, via la commande <tt>jupyter notebook</tt> ou toute autre méthode pertinente.
    </ul>
    
Il ne vous reste plus qu'à effectuer au fur et à mesure le travail demandé, qui consiste en général à remplir les cellules du présent notebook avec du code Python répondant à la question.

## 1. Principe du ray tracing


Pour déterminer la couleur de chacun des pixels de l'image, on poursuit le rayon partant de l'observateur et passant
par un pixel donné jusqu'à rencontrer un objet de la scène qui lui donne alors sa couleur. Si le rayon ne rencontre aucun objet on considère en général que le pixel sera noir.

<img src="Principe_raytracing.png" width="500"><br>
<center><i>fig.1 - Principe du ray tracing</i></center>

Lorsque la scène comporte plusieurs objets, ou des objets possédant plusieurs facettes comme le cube ci-dessus, on calcule l'intersection éventuelle entre le rayon et chacun des objets (ou chacune des facettes), et le pixel vu prendra la couleur correspondant au point d'intersection le plus proche.

Pour développer un ray tracer nous aurons donc besoin de plusieurs éléments :
- une scène comportant au moins un objet, à partir de laquelle il faudra générer une image,
- des objets dont il faudra calculer l'intersection avec les rayons,
- la possibilité de manipuler des vecteurs et des points en coordonnées 3D.

Nous nous intéresserons tout d'abord à un point technique : comment générer une image, puis
nous construirons un prototype de _ray tracer_ avec une scène comportant un seul objet, avant de lui porter diverses améliorations.

## 2. Génération d'une image

### 2.1 Le module numpy

Pour calculer des images, nous utiliserons le module numpy,
un module de Python destiné à manipuler des matrices ou tableaux multidimensionnels. Contrairement aux listes de python, les tableaux de numpy sont statiques c'est à dire de dimension fixe, et contiennent uniquement des éléments du même type. De ce fait, toutes les opérations sont hautement optimisées, ce qui permet d'effectuer beaucoup plus rapidement des calculs sur de gros volumes de données.

Numpy possède de nombreuses fonctions, mathématiques ou autres, opérant directement sur des tableaux.
Voici quelques exemples :

In [1]:
import numpy as np

a = np.array( [10,20,30,40,50] )
b = np.arange(1,6)

print('a = {}'.format(a))
print('b = {}'.format(b))

print('a+b = {}'.format(a+b))
print('a*b = {}'.format(a*b))
print('a^2 = {}'.format(a*a))

np.set_printoptions(precision=3)
print('sqrt(b) = {}'.format(np.sqrt(b)))

a = [10 20 30 40 50]
b = [1 2 3 4 5]
a+b = [11 22 33 44 55]
a*b = [ 10  40  90 160 250]
a^2 = [ 100  400  900 1600 2500]
sqrt(b) = [ 1.     1.414  1.732  2.     2.236]


### 2.2 Coordonnées des pixels

La technique consiste à créer tout d'abord le vecteur (tableau unidimensionnel) des abscisses et celui des ordonnées de l'ensemble des pixels de l'image.

Le code ci-dessous, largement commenté, effectue cette opération pour une image de 11x6 pixels.<br>
C'est bien sûr trop petit pour une image intéressante, mais cela permet d'afficher le résultat pour comprendre
l'allure des tableaux obtenus :

In [2]:
# Le module numpy permet facilement de manipuler des tableaux et
# d'effectuer des calculs sur un grand nombre de points
import numpy as np

# dimensions de l'image
w,h = 11,6

# Vecteur avec les abscisses de l'ensemble des pixels
# de l'image, ligne par ligne
x = np.tile(np.linspace(0, w-1, w), h)

# Vecteur avec les abscisses de l'ensemble des pixels
# de l'image, ligne par ligne
y = np.repeat(np.linspace(0, h-1, h), w)

# On prévient numpy que l'on veut afficher l'ensemble
# du contenu d'un tableau, et non uniquement un extrait
np.set_printoptions(threshold=np.inf)

# Tableau des abscisses
print(x)

# Tableau des ordonnées
print(y)

[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.   0.   1.   2.   3.
   4.   5.   6.   7.   8.   9.  10.   0.   1.   2.   3.   4.   5.   6.   7.
   8.   9.  10.   0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.   0.
   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.   0.   1.   2.   3.   4.
   5.   6.   7.   8.   9.  10.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.
  4.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.]


### 2.3 Valeur des pixels

L'opération suivante consiste à calculer une image pour chacun des pixels dont nous avons maintenant les coordonnées.

La couleur d'un pixel comprend trois composantes (rouge, verte, et bleue) dont nous exprimerons par convention la valeur entre 0 et 1 pour l'ensemble des calculs que nous effectuerons par la suite. La composition de différentes valeurs permet d'obtenir l'ensemble des couleurs. Voici quelques exemples :

$$\begin{align*}
(0,0,0) &\rightarrow \rm noir \\
(0.5,0.5,0.5) &\rightarrow \rm gris \\
(1,1,1) &\rightarrow \rm blanc \\
(1,0,0) &\rightarrow \rm rouge \\
(1,1,0) &\rightarrow \rm jaune \\
(1,0.647,0) &\rightarrow \rm orange \\
(0,1,0) &\rightarrow \rm vert\,clair \\
(0,0.5,0) &\rightarrow \rm vert\,foncé \\
...
\end{align*}$$

La génération d'une image consiste à créer 3 tableaux (un pour chacune des couleurs) avec la valeur souhaitée pour chacun des pixels.

Voici par exemple comment créer une image représentant une grille dont les lignes et les colonnes sont espacées de 10 pixels,
semblable à celle utilisée pour la figure 1.

In [3]:
# Le module PIL permet de générer un fichier image
# à partir d'un tableau de pixels
from PIL import Image

# L'image comprendra 221x161 pixels. x et y sont des
# tableaux monodimensionnels de 221x161 = 35561 éléments
w,h = 221,161
x = np.tile(np.linspace(0, w-1, w), h)
y = np.repeat(np.linspace(0, h-1, h), w)

# En admirant la concision du code que permet numpy on construit
# un tableau de booléens à 35561 éléments, presque tous faux,
# mais vrais sur les lignes et colonnes multiples de 10
grid = (x % 10 == 0) | (y % 10 == 0)

# On génère un tableau de 35561 valeurs (0 ou 1)
# La fonction np.where permet de choisir entre ces deux valeurs
# en fonction des valeurs true ou false du tableau grid :
# true -> 0, false -> 1
b = np.where(grid,0,1)

# Dans le cas particulier de cette image nous aurons
# uniquement du noir et du blanc. Les trois composantes
# rouge, verte et bleue de chacun des pixels auront la même valeur.
r = g = b

# Le code ci-dessous crée le fichier image.
# Il est un tantinet technique mais il n'est pas nécessaire de le
# comprendre dans le détail, et nous n'aurons pas à y revenir par la suite.
#
# Voici néanmoins le détail de la démarche pour ceux que cela intéresse :
# - on exprime les couleurs entre 0 et 255, valeurs liées au format d'image souhaité,
# - on restructure les tableaux sous forme de matrices de dimension w,h (reshape)
# - on convertit (astype) les valeurs en entiers 8 bits (np.uint8)
# - on génère une image monochrome (Image.fromarray) pour chacune des composantes
# - on crée une liste de ces trois images (rgb_img = [... for c in (rgb)])
# - on écrit un fichier nommé grid.png à partir de la liste rgb_img
#
# Exercice pour comprendre : détailler ce code en passant par des variables
# intermédiaires à chacune des étapes.
#
name = 'grid'
rgb_img = [
    Image.fromarray((255 * c.reshape((h,w))).astype(np.uint8), "L")
    for c in (r,g,b) ]
Image.merge("RGB", rgb_img).save("{}.png".format(name))

<img src="ref_grid.png" width="221"><br>
<center><i>fig.2 - <a href="grid.png">grid.png</a></i></center>

### _Travail à effectuer :_

Afin de conserver un exemple opérationnel il est très fortement suggéré de ne pas modifier le code ci-dessus, mais de développer votre propre code à l'endroit suggéré ci-dessous, en ajoutant éventuellement de nouvelles cellules au notebook au fur et à mesure du travail demandé.

Après avoir copié le code pour le modifier, il ne faut pas oublier de changer également le nom de l'image (name = 'grid') pour éviter de l'écraser à chaque fois.

__Q1. A faire en TD :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
&Agrave; partir des dernières lignes de l'exemple ci-dessus, écrire une fonction <i>draw_image(name, w, h, r, g, b)</i> permettant de créer un fichier image.
Utiliser ensuite cette fonction pour créer une grille dont les couleurs ne sont pas le noir et le blanc.
</div>

In [4]:
# votre code ici

<table style="border-color:transparent; border-spacing:50px 0; border-collapse:separate"><tr>
<td style="border:none"><img src="ref_colored_grid.png" width="221"><br>
<center><i>exemple de résultat</i></center>
</td>
<td style="border:none"><img src="colored_grid.png" width="221"><br>
<center><i>votre résultat</i></center>
</td>
</tr><tr>
<td style="border:none; text-align:center" colspan="2"><i>question 1. <a href="colored_grid.png">colored_grid.png</a></i></td>
</tr></table>

__N.B.__ Une fois l'image créée et affichée par le notebook dans la cellule ci-dessus, les modifications ultérieures de l'image (par réexécution du code modifié, par exemple) ne seront pas immédiatement visibles. Pour obliger le notebook à recharger l'image, il faut :<br>
&bull; sauvegarder les dernière modifications (icône disquette en haut à gauche de la barre d'icônes),<br>
&bull; recharger la page dans le navigateur, ce qui a malheureusement également pour effet de redémarrer le noyau python, et oblige à réexécuter toutes les cellules pour récupérer les différentes variables initialisées plus haut.

En cas de modifications répétées de l'image, il est donc utile de la visualiser dans une fenêtre de navigateur dédiée que l'on pourra alors recharger à volonté  sans perdre l'avancement dans le notebook. Afin de faciliter cette opération, dans toute la suite on pourra visualiser l'image dans une autre fenêtre par un simple clic sur la légende.


__Q2. Pour les plus rapides :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Créer une grille dont la cellule centrale est colorée (comme sur l'exemple ci-dessous.).
</div>

In [5]:
# votre code ici

<table style="border-color:transparent; border-spacing:50px 0; border-collapse:separate"><tr>
<td style="border:none"><img src="ref_grid_with_pixel.png" width="221"><br>
<center><i>exemple de résultat</i></center>
</td>
<td style="border:none"><img src="grid_with_pixel.png" width="221"><br>
<center><i>votre résultat</i></center>
</td>
</tr><tr>
<td style="border:none; text-align:center" colspan="2"><i>question 2. <a href="grid_with_pixel.png">grid_with_pixel.png</a> </i></td>
</tr></table>

__Q3. Pour comprendre comment ça marche :__ (en TD ou plus tard)
<div style="background-color:#eef;padding:10px;border-radius:3px">
Créer d'autres images - carrés, rectangles, cercles...
</div>

In [6]:
# votre code ici

<table style="border-color:transparent; border-spacing:50px 0; border-collapse:separate"><tr>
<td style="border:none"><img src="ref_jp_flag.png" width="221"><br>
<center><i>exemple de résultat</i></center>
</td>
<td style="border:none"><img src="jp_flag.png" width="221"><br>
<center><i>votre résultat</i></center>
</td>
</tr><tr>
<td style="border:none; text-align:center" colspan="2"><i>question 3. <a href="jp_flag.png">jp_flag.png</a></i></td>
</tr></table>

## 3. Prototype de ray tracer


### 3.1 Vecteurs 3D

Nous avons vu dans la section précédente que _numpy_ nous permet d'effectuer très simplement des calculs sur un tableau de valeurs, qui nécessiteraient autrement (i.e. sans _numpy_) d'effectuer avec python une boucle sur l'ensemble des valeurs du tableau pour leur appliquer le traitement souhaité.

Au-delà d'une meilleure lisibilité du code, les calculs effectués de cette manière via _numpy_ sont __environ 10 fois plus rapides__ que les mêmes calculs effectués à l'aide de boucles python.

Forts de ce constat, et devant la nécessité de manipuler des coordonnées 3D, _vec3_ est une classe permettant de travailler aisément avec des tableaux de points dans l'espace, ou de vecteurs 3D, ou de couleurs (puisqu'une couleur comprend également 3 composantes réelles).

Un objet de la classe _vec3_ comprend trois composantes (x,y,z) qui peuvent chacune correspondre à une valeur réelle, ou alors à un tableau numpy (comme par exemple X et Y dans la question précédente).

In [7]:
# Utilisé pour tester si un paramètre est un nombre ou non
import numbers

# N.B. les méthodes dont le nom comporte des __ sont des méthodes spéciales dont
# le nom est réservé, et correspondent à une surcharge d'opérateur. Elles sont 
# automatiquement appelées par python en fonction de l'opération effectuée.
#
# Pour effectuer par exemple une addition entre un vec3 (opérande de gauche) et une
# valeur quelconque (opérande de droite), python appelle la méthode __add__ de vec3
# avec self correspondant au vec3 (opérande de gauche) et other à l'opérande de droite. 
class vec3():
    def __init__(self, x, y, z):
        (self.x, self.y, self.z) = (x, y, z)

    # multiplication par une constante : v * k (attention k * v génère une erreur)
    def __mul__(self, other):
        return vec3(self.x * other, self.y * other, self.z * other)

    # addition de deux vecteurs : v1 + v2
    def __add__(self, other):
        return vec3(self.x + other.x, self.y + other.y, self.z + other.z)

    # soustraction de deux vecteurs : v1 - v2
    def __sub__(self, other):
        return vec3(self.x - other.x, self.y - other.y, self.z - other.z)

    # produit scalaire : v1.dot(v2) = v1.v2
    def dot(self, other):
        return (self.x * other.x) + (self.y * other.y) + (self.z * other.z)

    # Carré de la valeur absolue : abs2(v) = v.v
    def abs2(self):
        return self.dot(self)
    
    # valeur absolue : abs(v) = sqrt(v.v)
    def __abs__(self):
        return np.sqrt(self.dot(self))

    # normalisation : v = v.norm() * sqrt(v.v)
    def norm(self):
        mag = abs(self)
        return self * (1.0 / np.where(mag == 0, 1, mag))

    # composantes : v.components() = (x,y,z)
    def components(self):
        return (self.x, self.y, self.z)

    # affichage : print(v)
    def __str__(self):
        f = "{}({:.3f}, {:.3f}, {:.3f})" if isinstance(self.x, numbers.Number) else "{}({}, {}, {})"
        return f.format(self.__class__.__name__,*self.components())
    
# Une couleur est exprimée grâce à trois composantes réelles
# On définit rgb comme un synonyme de vec3
class rgb(vec3):
    pass

# Carré de la valeur absolue
abs2 = vec3.abs2

__Q4. Tester le fonctionnement de vec3 :__ 
<div style="background-color:#eef;padding:10px;border-radius:3px">
Créer les vecteurs 3D : $V_1 = (1,0,0) \qquad V_2 = (0,1,0) \qquad V_3 = (0,0,1) \qquad V_4 = (1,1,0) \qquad V_5 = (1,1,1)$<br>
et vérifier que les opérations définies ci-dessus (multiplication par un scalaire, addition, soustraction,
produit scalaire, valeur absolue, normalisation), fournissent le résultat escompté.
</div>

In [8]:
# votre code ici

On convient à partir de maintenant de situer le plan de l'image en $Z = 0$, et de définir les unités de la scène avec
les abscisses du rectangle de l'image allant de $-1$ à $+1$,
et les ordonnées de -w/h à +w/h en fonction de la géométrie de l'image :

<img src="raytrace_params.png" width="500"><br>
<center style="margin-top -1em"><i>fig.3 - Paramètres de raytrace</i></center>

__Q5. Coordonnées des pixels de l'image :__ 
<div style="background-color:#eef;padding:10px;border-radius:3px">
Afin de tirer parti des possibilités de _numpy_ permettant d'effectuer les calculs en parallèle pour l'ensemble des pixels de l'image, calculer $\rm Q$, objet _vec3_ dont les composantes correspondent respectivement aux abscisses $\rm X$, aux ordonnées $\rm Y$ et à la profondeur $\rm Z$ des pixels de l'image pour une image de 300x400 pixels.
</div>

In [9]:
# votre code ici

__Q6. Rayons primaires :__ 
<div style="background-color:#eef;padding:10px;border-radius:3px">
On appelle _rayons primaires_ les droites orientées partant de l'observateur et passant par les pixels de l'image. Calculer $\rm D$, objet _vec3_ avec l'ensemble des vecteurs directeurs normés correspondant aux rayons primaires pour une image de 400x300 pixels avec un observateur situé en ${\rm O} = (0,0.1,-10)$
</div>

In [10]:
# votre code ici

### 3.2 La sphère

Afin de peupler la scène, définissons l'objet le plus simple : la sphère. Le principe du _ray tracing_ (cf. fig.1) nous suggère que la classe _Sphere_ devra être munie d'une méthode permettant de calculer son intersection avec un rayon primaire.

Pour cela, considérons une sphère $S$ de centre $\rm C$ et de rayon $R$.

Avec la fonction $abs2()$ prenant le sens de la méthode _abs2()_ de _vec3_,
la sphère est constituée des points $\rm P$ tels que :
$abs2(\overrightarrow{\rm CP}) = R^2$

Les rayons primaires passent par $\rm O$, et ont pour vecteur directeur $\overrightarrow{\rm D}$.<br>
Ils sont donc
constitués des points $\rm P$ pour lesquels il existe un coefficient $h$ tel que :
$(\overrightarrow{\rm OP}) = h\,\overrightarrow{\rm D}$

En injectant dans l'équation de la sphère : $abs2(\overrightarrow{\rm CP}) = abs2(\overrightarrow{\rm CO} + \overrightarrow{\rm OP}) = abs2(\overrightarrow{\rm CO} + h\,\overrightarrow{\rm D}) = R^2$

En se souvenant que $abs2(\overrightarrow{\rm X}) = \overrightarrow{\rm X}.\overrightarrow{\rm X}$ et en développant,
il vient :
$\overrightarrow{\rm CO}.\overrightarrow{\rm CO} + 2h\,\overrightarrow{\rm CO}.\overrightarrow{\rm D} + h^2\,\overrightarrow{\rm D}.\overrightarrow{\rm D} = R^2$

Que l'on peut simplifier en notant que $\overrightarrow{\rm D}$ est normé, pour obtenir une équation du second degré en $h$ :

$$h^2 + 2\,h\,\overrightarrow{\rm D}.\overrightarrow{\rm CO} + \overrightarrow{\rm CO}.\overrightarrow{\rm CO} - R^2 = 0$$

Comme $h$ représente la distance entre l'observateur et le point d'intersection le long du rayon primaire, la méthode _intersect_ renvoie la plus petite des solutions de cette équation lorsqu'il en existe une. En l'absence de solution (i.e. pas de point d'intersection) la méthode renvoie un nombre très grand _(faraway)_ assimilé à l'infini. 

In [11]:
class Sphere():
    def __init__(self, center, r):
        self.c = center
        self.r = r
    
    # O : source du rayon
    # D : direction du rayon
    def intersect(self, O, D, faraway):
        b = 2 * D.dot(O - self.c)
        c = abs2(O-self.c) - (self.r * self.r)
        disc = (b ** 2) - (4 * c)
        sq = np.sqrt(np.maximum(0, disc))
        h0 = (-b - sq) / 2
        h1 = (-b + sq) / 2
        h = np.where((h0 > 0) & (h0 < h1), h0, h1)
        pred = (disc > 0) & (h > 0)
        return np.where(pred, h, faraway)

__Q7. Calcul des points d'intersection :__ 
<div style="background-color:#eef;padding:10px;border-radius:3px">
En utilisant la fonction <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.extract.html">_np.extract_</a> de _numpy_, déterminer le nombre de pixels qu'occupe la sphère de rayon $0.5$ centrée
en $(0,0.5,5)$ sur une image de 400x300 pixels positionnée comme précédemment, avec l'observateur en $(0,0.1,−10)$.
</div>

In [12]:
# votre code ici

### 3.3 La scène

La classe _Scene_ implémente le modèle de la scène :

- le constructeur attend le nom de la scène (qui sera utilisé pour le nom du fichier image créé), un objet (une sphère en l'état actuel des choses) et la valeur de l'infini (1e39 par exemple),

- _initialize()_ prend les dimensions de l'image et la position de l'observateur et calcule les coordonnées des pixels (tableau $\rm Q$),

- _trace()_ note l'heure puis effectue le calcul de l'image grâce à un appel à la méthode _raytrace_, et affiche la durée du calcul,

- <i>save_image()</i> crée le fichier image,

- _raytrace()_ calcule les points d'intersection de l'objet avec les rayons primaires, puis renvoie un tableau de couleurs, c'est-à-dire un triplet de valeurs (r,g,b) pour chacun des pixels de l'image. Pour ce premier jet on se contente de représenter la distance de chacun des points de l'objet à l'oeil de l'observateur.

In [13]:
# Utilisé pour comptabiliser les temps de calcul
import time

# Définition de la scène
class Scene:

    # Crée une scène avec un objet passé en paramètre
    # - le nom de la scène sert pour le nom du fichier
    # - FARAWAY représente une distance infinie, utilisée pour
    #   les points n'ayant aucune intersection avec la sphère
    def __init__(self,name,obj,faraway=1.0e39):
        self.obj = obj
        self.name = name
        self.FARAWAY = faraway
        
        # cf. initialize()
        self.w = 0
        self.h = 0
        self.Q = None

        # cf. trace()
        self.data = None
        
        # cf. save_image
        self.rgb_img = None

    # Calcule les coordonnées x,y des pixels dans les unités de la scène.
    # - w et h sont les dimensions de l'image
    # - E correspond à la position 3D de l'observateur
    #     on suppose que la caméra se trouve par défaut en (0,0.1,-10)
    # Les coordonnées des pixels de l'image sont définies de la manière suivante :
    # - le plan de l'image est situé en z = 0,
    # - les abscisses du rectangle de l'image vont de -1 à +1,
    # - les ordonnées de -w/h à +w/h en fonction de la géométrie de l'image 
    def initialize(self,w = 400,h = 300,E = None):
        self.camera = E if E else vec3(0., 0.1, -10.)
        self.w = w
        self.h = h
        
        r = float(self.w) / self.h
        x = np.tile(np.linspace(-1., 1., self.w), self.h)
        y = np.repeat(np.linspace(1./r, -1./r, self.h), self.w)
        
        # position 3D de l'ensemble des pixels de la scène
        self.Q = vec3(x, y, 0)

        # pour une interface fluide permettant de chaîner les appels
        return self

    # Calcule les couleurs r,g,b de l'image via l'appel à la méthode raytrace
    # et affiche le temps qui a été nécessaire au calcul
    # Les paramètres passés à la méthode raytrace sont :
    # - la position 3D de l'observateur
    # - le tableau des vecteurs directeurs de tous les rayons depuis l'observateur
    #   vers chacun des pixels de l'image (cf. fig. 3 - paramètres de raytrace)
    def trace (self,silent = False):
        t0 = time.perf_counter()
        self.data = self.raytrace(self.camera, (self.Q - self.camera).norm())
        if ( not silent ):
            print("{}.png took {:.3f}s".format(self.name,time.perf_counter() - t0))
        return self

    # Crée le fichier image
    def save_image(self):
        self.rgb_img = [
            Image.fromarray((255 * np.clip(c,0,1).reshape((self.h,self.w))).astype(np.uint8), "L")
            for c in self.data.components() ]
        Image.merge("RGB", self.rgb_img).save("{}.png".format(self.name))
        return self

    # Renvoie la couleur obtenue pour l'ensemble des pixels définis par :
    # O : l'origine du rayon
    # D : le vecteur directeur de l'ensemble des rayons
    #
    # N.B. Pour l'instant on se contente de renvoyer une nuance de gris (r = g = b)
    # avec une valeur entre 0 et 1 correspondant à la distance de l'objet
    def raytrace(self, O, D):

        # renvoie un tableau avec la distance entre l'observateur et le point
        # d'intersection avec l'objet de la scène, pour l'ensemble des rayons,
        # ou FARAWAY pour les rayons qui ne touchent pas l'objet
        distances = self.obj.intersect(O, D, self.FARAWAY)

        # Calcul du minimum des distances
        nearest = np.min(distances)
        
        # masked contient les mêmes informations que distances, mais
        # avec la distance des pixels sans intersection avec un objet
        # forcée à 0 (au lieu de FARAWAY) afin de pouvoir calculer le max
        mask = (distances != self.FARAWAY)
        masked = np.where(mask, distances, 0)

        # Calcul du maximum des distances (hors FARAWAY - d'où l'utilisation du masque)
        farthest = np.max(masked)
        
        # Normalisation des distances entre 0 et 1, en évitant de diviser par 0
        scaled = np.where(mask, (farthest - masked)/(farthest - nearest), 0);

        # Nuances de gris en fonction de la distance
        return rgb(scaled, scaled, scaled)

### 3.4 Le résultat :

__Q8. Programme principal :__ 
<div style="background-color:#eef;padding:10px;border-radius:3px">
Ecrire à l'aide des classes précédentes le programme permettant de générer l'image d'une sphère de rayon $0.5$ située en $(0,0,3)$, tous les autres paramètres prenant leur valeur par défaut (taille de l'image, position de l'observateur, valeur de l'infini). Noter le temps de calcul (typiquement une petite fraction de seconde) et vérifier l'image obtenue.
</div>

In [14]:
# votre code ici

<table style="border-color:transparent; border-spacing:50px 0; border-collapse:separate"><tr>
<td style="border:none"><img src="ref_distances.png" width="221"><br>
<center><i>résultat attendu</i></center>
</td>
<td style="border:none"><img src="distances.png" width="221"><br>
<center><i>votre résultat</i></center>
</td>
</tr><tr>
<td style="border:none; text-align:center" colspan="2"><i>question 8. <a href="distances.png">distances.png</a></i></td>
</tr></table>

__Q9. Voir en couleur :__ (pour les plus rapides)
<div style="background-color:#eef;padding:10px;border-radius:3px">
Ajouter un attribut pour donner une couleur à la sphère. Modifier le calcul pour générer une image <i>colored_sphere.png</i>.
</div>

In [15]:
# votre code ici

<table style="border-color:transparent; border-spacing:50px 0; border-collapse:separate"><tr>
<td style="border:none"><img src="ref_colored_sphere.png" width="221"><br>
<center><i>exemple de résultat</i></center>
</td>
<td style="border:none"><img src="colored_sphere.png" width="221"><br>
<center><i>votre résultat</i></center>
</td>
</tr><tr>
<td style="border:none; text-align:center" colspan="2"><i>question 9. <a href="colored_sphere.png">colored_sphere.png</a></i></td>
</tr></table>

__Q10. Scène composée :__ (pour s'entrainer)
<div style="background-color:#eef;padding:10px;border-radius:3px">
Modifier la scène pour accepter une liste d'objets (et non plus un seul).
</div>

In [16]:
# votre code ici

<table style="border-color:transparent; border-spacing:50px 0; border-collapse:separate"><tr>
<td style="border:none"><img src="ref_colored_spheres.png" width="221"><br>
<center><i>exemple de résultat</i></center>
</td>
<td style="border:none"><img src="colored_spheres.png" width="221"><br>
<center><i>votre résultat</i></center>
</td>
</tr><tr>
<td style="border:none; text-align:center" colspan="2"><i>question 10. <a href="colored_spheres.png">colored_spheres.png</a></i></td>
</tr></table>

__N.B.__ La sphère située à l'arrière est très sombre parce que la couleur a été modulée en fonction de la distance, entre le point le plus proche de la sphère la plus proche et le point le plus lointain de la sphère la plus lointaine.

Pour un meilleur rendu il va falloir modifier le calcul de la couleur en un point donné des objets pour utiliser un modèle d'éclairage réaliste.

## 4. Vers un rendu réaliste

### 4.1 Sources de lumière &ndash; _Lambert shading_

Pour une résultat plus réaliste, il faut calculer la couleur de chacun des points en tenant compte de l'éclairage de la scène. Pour cela il nous faut des sources de lumière :

In [17]:
class LightSource():
    def __init__(self,position,color):
        self.position = position
        self.color = color

La couleur en un point d'un objet est ensuite calculée en multipliant la couleur de l'objet avec celle de la source de lumière, tout en modulant l'amplitude en fonction de l'angle sous lequel l'objet est éclairé. L'éclairage sera maximal si le rayon de lumière incident est normal à la surface au point considéré, beaucoup plus faible en cas d'éclairage rasant.

La méthode _light()_ effectue ce calcul et nous renvoie la couleur de la sphère au point ${\rm M} = {\rm O} + d\,\overrightarrow{\rm D}$ :

In [18]:
class LightedSphere(Sphere):
    
    def __init__(self, center, r, color):
        Sphere.__init__(self, center, r)
        self.color = color

    # renvoie la couleur de la sphère au point M
    def diffusecolor(self, M):
        return self.color

    # renvoie le vecteur normal normalisé au point M 
    def normal(self, M):
        return (M - self.c)*(1./self.r)

    # light : renvoie la couleur en un point donné d'un objet
    #   O : Origine du rayon incident
    #   D : Direction normalisée du rayon incident
    #   d : distance depuis l'origine au point considéré
    #   scene : scène à laquelle appartient l'objet
    def light(self, O, D, d, scene):
        M = (O + D * d)                 # point sur l'objet
        N = self.normal(M)              # normale au point M
        
        # Couleur locale au point M
        objcolor = self.diffusecolor(M)
 
        # direction sous laquelle M voit la source de lumière
        toL = (scene.source.position - M).norm()

        # produit scalaire entre la direction de la source de lumière
        # et la normale en M, limité aux valeurs positives
        lv = np.maximum(N.dot(toL), 0)

        # Lambert shading (réflexion diffuse)
        color = objcolor.mul(scene.source.color) * lv

        return color

    
class rgb(vec3):
    
    # multiplication composant à composante
    def mul(self, other):
        return rgb(self.x * other.x, self.y * other.y, self.z * other.z)

Il reste maintenant à modifier la fonction _raytrace()_ pour appeler la méthode _light()_ afin d'obtenir la couleur d'un pixel :

In [19]:
# Pour la fonction reduce
import functools

class LightedScene(Scene):

    def __init__(self,name,source,objects,faraway=1.0e39):
        Scene.__init__(self,name,None,faraway)
        self.source = source
        self.objects = objects

    def raytrace(self, O, D):

        # tableau des distances pour chacun des objets
        distances = [obj.intersect(O, D, self.FARAWAY) for obj in self.objects]

        # tableau de la distance minimale par pixel
        nearest = functools.reduce(np.minimum, distances)
        
        # on part du noir
        color = rgb(0, 0, 0)
        
        # boucle sur les objets de la scène
        for (s, d) in zip(self.objects, distances):
            
            # on tient compte de la couleur de l'objet
            # s'il y a un point d'intersection avec le rayon primaire (nearest != FARAWAY)
            # et que l'objet n'est pas caché par un autre objet (d == nearest)
            color += s.light(O, D, d, scene) * (nearest != self.FARAWAY) * (d == nearest)
   
        return color

__Q11. Lambert shading :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Créer une scène et générer une image comprenant plusieurs sphères et une source de lumière. 
</div>

In [20]:
# votre code ici

<table style="border-color:transparent; border-spacing:50px 0; border-collapse:separate"><tr>
<td style="border:none"><img src="ref_lighted_spheres.png" width="221"><br>
<center><i>exemple de résultat</i></center>
</td>
<td style="border:none"><img src="lighted_spheres.png" width="221"><br>
<center><i>votre résultat</i></center>
</td>
</tr><tr>
<td style="border:none; text-align:center" colspan="2"><i>question 11. <a href="lighted_spheres.png">lighted_spheres.png</a></i></td>
</tr></table>

Pour adoucir la dureté des ombres, il suffit de multiplier les sources de lumière :

In [21]:
class IlluminatedScene(LightedScene):

    def __init__(self,name,sources,objects,faraway=1.0e39):
        Scene.__init__(self,name,None,faraway)
        self.sources = sources
        self.objects = objects
        
class IlluminatedSphere(LightedSphere):
    
    def light(self, O, D, d, scene):
        M = (O + D * d)                 # point sur l'objet
        N = self.normal(M)              # normale au point M
        objcolor = self.diffusecolor(M) # couleur de l'objet au point M

        # on part du noir
        color = rgb(0,0,0)
       
        # boucle sur les sources de lumière
        for s in scene.sources:
            toL = (s.position - M).norm()
            lv = np.maximum(N.dot(toL), 0)

            # Lambert shading (réflexion diffuse)
            color += objcolor.mul(s.color) * lv

        return color

__Q12. Scène à plusieurs sources de lumière :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Créer une scène et générer une image comprenant plusieurs sphères et plusieurs sources de lumière. 
</div>

In [22]:
# votre code ici

<table style="border-color:transparent; border-spacing:50px 0; border-collapse:separate"><tr>
<td style="border:none"><img src="ref_illuminated_spheres.png" width="221"><br>
<center><i>exemple de résultat</i></center>
</td>
<td style="border:none"><img src="illuminated_spheres.png" width="221"><br>
<center><i>votre résultat</i></center>
</td>
</tr><tr>
<td style="border:none; text-align:center" colspan="2"><i>question 12. <a href="illuminated_spheres.png">illuminated_spheres.png</a></i></td>
</tr></table>

### 4.2 &Eacute;clairage ambiant

La dureté des ombres peut également être adoucie en ajoutant de l'éclairage d'ambiance dans la valeur renvoyée par la méthode _light()_. La couleur de l'éclairage ambiant sera un attribut de la scène :

In [23]:
class AmbientScene(IlluminatedScene):

    def __init__(self,name,sources,objects,ambient,faraway=1.0e39):
        IlluminatedScene.__init__(self,name,sources,objects,faraway)
        self.ambient = ambient
    
class AmbientSphere(IlluminatedSphere):
    
    def light(self, O, D, d, scene):
        M = (O + D * d)                 # point sur l'objet
        N = self.normal(M)              # normale au point M
        objcolor = self.diffusecolor(M) # couleur de l'objet au point M

        # éclairage ambiant
        color = objcolor.mul(scene.ambient)
       
        # boucle sur les sources de lumière
        for s in scene.sources:
            toL = (s.position - M).norm()
            lv = np.maximum(N.dot(toL), 0)

            # Lambert shading (réflexion diffuse)
            color += objcolor.mul(s.color) * lv

        return color

__Q13. Scène avec éclairage ambiant :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Créer une scène et générer une image comprenant plusieurs sphères, plusieurs sources de lumière et de l'éclairage ambiant. 
</div>

In [24]:
# votre code ici

<table style="border-color:transparent; border-spacing:50px 0; border-collapse:separate"><tr>
<td style="border:none"><img src="ref_ambient_spheres.png" width="221"><br>
<center><i>exemple de résultat</i></center>
</td>
<td style="border:none"><img src="ambient_spheres.png" width="221"><br>
<center><i>votre résultat</i></center>
</td>
</tr><tr>
<td style="border:none; text-align:center" colspan="2"><i>question 13. <a href="ambient_spheres.png">ambient_spheres.png</a></i></td>
</tr></table>

### 4.3 Plan quadrillé

Pour éviter que les sphères ne flottent dans l'espace, on peut simuler un plan placé sous les sphères à l'aide d'une sphère de très grand diamètre. L'effet quadrillé est obtenu à l'aide de la méthode _diffuse()_ des sphères en renvoyant une couleur différente en fonction des coordonnées de M.

In [25]:
class CheckeredSphere(AmbientSphere):
    def diffusecolor(self, M):
        checker = (((M.x * 2 - 1000.5).astype(int) % 2) == ((M.z * 0.2 - 1000.25).astype(int) % 2))
        return rgb(
            np.where(checker,self.color[0].x,self.color[1].x),
            np.where(checker,self.color[0].y,self.color[1].y),
            np.where(checker,self.color[0].z,self.color[1].z)
        )

__Q14. Scène avec plan quadrillé :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Ajouter un plan quadrillé sous les sphères de votre scène, en notant que la sphère quadrillée devra être créée en passant un tuple de couleurs au lieu d'une couleur unique.
<br><br>
__N.B.__ Si vous vous interrogez sur l'expression utilisée par la méthode _diffusecolor()_ pour calculer le quadrillage, il n'est pas interdit de jouer en modifiant l'expression pour mieux comprendre à quoi elle correspond.
</div>

In [26]:
# votre code ici

<table style="border-color:transparent; border-spacing:50px 0; border-collapse:separate"><tr>
<td style="border:none"><img src="ref_checkered_scene.png" width="221"><br>
<center><i>exemple de résultat</i></center>
</td>
<td style="border:none"><img src="checkered_scene.png" width="221"><br>
<center><i>votre résultat</i></center>
</td>
</tr><tr>
<td style="border:none; text-align:center" colspan="2"><i>question 14. <a href="checkered_scene.png">checkered_scene.png</a></i></td>
</tr></table>

### 4.4 Ombres portées

Pour observer des ombres portées il suffit de ne pas prendre en compte la couleur d'un objet lorsque ce point ne voit pas la source de lumière. Il suffit pour cela de lancer un rayon depuis l'objet vers la source de lumière et de vérifier s'il rencontre ou non d'autres objets.

Pour cela, on cherche s'il existe une intersection entre la demi-droite partant du point considéré vers la source de lumière et chacun des objets de la scène. Ce calcul est effectué grâce à la méthode _intersect()_ dont nous disposons déjà.

In [27]:
 class ShadowedSphere(AmbientSphere):
    
    def light(self, O, D, d, scene):
        M = (O + D * d)                 # point sur l'objet
        N = self.normal(M)              # normale au point M
        nudged = M + N * .0001          # point légèrement éloigné sur la normale en M pour éviter de se voir soi-même
        
        objcolor = self.diffusecolor(M)
        color = objcolor.mul(scene.ambient)

        # boucle sur les sources de lumière
        for s in scene.sources:
            toL = (s.position - M).norm()   # direction sous laquelle M voit la source de lumière
            distL = abs(s.position - M)     # distance à laquelle M voit la source de lumière
            
            # M voit-il la source de lumière ?
            light_distances = [obj.intersect(nudged, toL, scene.FARAWAY) for obj in scene.objects]
            light_nearest = functools.reduce(np.minimum, light_distances)
            seelight = distL < light_nearest

            # Lambert shading (réflexion diffuse)
            lv = np.maximum(N.dot(toL), 0)
            color += objcolor.mul(s.color) * lv * seelight

        return color

# On redéfinit la classe CheckeredSphere pour hériter de la méthode light avec ombres portées
class ShadowedCheckeredSphere(ShadowedSphere):
    def diffusecolor(self, M):
        checker = (((M.x * 2 - 1000.5).astype(int) % 2) == ((M.z * 0.2 - 1000.25).astype(int) % 2))
        return rgb(
            np.where(checker,self.color[0].x,self.color[1].x),
            np.where(checker,self.color[0].y,self.color[1].y),
            np.where(checker,self.color[0].z,self.color[1].z)
        )

__Q15. Scène avec ombres portées :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Créer une scène mettant en évidence des ombres portées.
</div>

In [28]:
# votre code ici

<table style="border-color:transparent; border-spacing:50px 0; border-collapse:separate"><tr>
<td style="border:none"><img src="ref_shadowed_scene.png" width="221"><br>
<center><i>exemple de résultat</i></center>
</td>
<td style="border:none"><img src="shadowed_scene.png" width="221"><br>
<center><i>votre résultat</i></center>
</td>
</tr><tr>
<td style="border:none; text-align:center" colspan="2"><i>question 15. <a href="shadowed_scene.png">shadowed_scene.png</a></i></td>
</tr></table>

### 4.5 Surfaces brillantes - _Phong shading_

Les résultats obtenus jusqu'à présent donnent l'impression de surfaces mates. Pour obtenir des surfaces brillantes il faut tenir compte d'une réflexion dite spéculaire où la luminance de la surface est maximale dans la direction dans laquelle se réfléchit la lumière de la source, tout en diminuant rapidement lorsqu'on s'éloigne de cette direction privilégiée.

Il existe divers modèles pour simuler ce comportement, dont la méthode de Phong (Phong shading) ou une approximation d'icelle (Blinn-Phong). Pour plus d'info sur le principe de ces méthodes, cf.
<a href="https://en.wikipedia.org/wiki/Phong_reflection_model">Phong</a> et
<a href="https://en.wikipedia.org/wiki/Blinn%E2%80%93Phong_shading_model">Blinn-Phong</a> sur wikipédia.


In [29]:
class ShinySphere(ShadowedSphere):

    def __init__(self, center, r, color, lambert, specular, phong):
        Sphere.__init__(self, center, r)
        self.color = color
        self.lambert = lambert
        self.specular = specular
        self.phong = phong
        
    def light(self, O, D, d, scene):
        M = (O + D * d)                 # point sur l'objet
        N = self.normal(M)              # normale au point M
        toO = (scene.camera - M).norm() # vecteur depuis l'observateur vers le point M
        nudged = M + N * .0001          # point légèrement éloigné sur la normale en M pour éviter de se voir soi-même
        
        objcolor = self.diffusecolor(M)
        color = objcolor.mul(scene.ambient)

        # boucle sur les sources de lumière
        for s in scene.sources:
            toL = (s.position - M).norm()     # direction sous laquelle M voit la source de lumière
            distL = abs(s.position - M)       # distance à laquelle M voit la source de lumière
            
            # M voit-il la source de lumière ?
            light_distances = [obj.intersect(nudged, toL, scene.FARAWAY) for obj in scene.objects]
            light_nearest = functools.reduce(np.minimum, light_distances)
            seelight = distL < light_nearest

            # Lambert shading (réflexion diffuse)
            lv = np.maximum(N.dot(toL), 0)
            color += objcolor.mul(s.color) * self.lambert * lv * seelight
            
            # Blinn-Phong (réflexion spéculaire)
            phong = N.dot((toL + toO).norm())
            color += s.color * self.specular * np.power(np.clip(phong, 0, 1), self.phong) * seelight

        return color

# On redéfinit la classe CheckeredSphere pour hériter de la méthode light avec Phong
class ShinyCheckeredSphere(ShinySphere):
    def diffusecolor(self, M):
        checker = (((M.x * 2 - 1000.5).astype(int) % 2) == ((M.z * 0.2 - 1000.25).astype(int) % 2))
        return rgb(
            np.where(checker,self.color[0].x,self.color[1].x),
            np.where(checker,self.color[0].y,self.color[1].y),
            np.where(checker,self.color[0].z,self.color[1].z)
        )

__Q16. Scène avec réflexion spéculaire :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Créer une scène comportant des réflexions spéculaires.
</div>

In [30]:
# votre code ici

<table style="border-color:transparent; border-spacing:50px 0; border-collapse:separate"><tr>
<td style="border:none"><img src="ref_shiny_scene.png" width="221"><br>
<center><i>exemple de résultat</i></center>
</td>
<td style="border:none"><img src="shiny_scene.png" width="221"><br>
<center><i>votre résultat</i></center>
</td>
</tr><tr>
<td style="border:none; text-align:center" colspan="2"><i>question 16. <a href="shiny_scene.png">shiny_scene.png</a></i></td>
</tr></table>

### 4.6 Réflexions

Pour encore plus de réalisme, on peut représenter des surfaces réfléchissantes. Pour ceci, il suffit de placer l'oeil (ou la caméra) au point M sur l'objet, en regardant dans la direction du rayon réfléchi. Ce rayon est symétrique du rayon incident (le regard) par rapport à la normale au point M :
<br>
<img src="reflexion.png">
<br>
Dans notre cas, le rayon incident est $\vec{D} = -\vec{L}$ et la direction dans laquelle "regarde" le rayon réfléchi est donc :

$$ \vec{D}_{réflexion} = \vec{D} - 2 \,(\vec{D}.\vec{N})\,\vec{N}$$

A partir de là il est très simple d'obtenir des images extrêmement réalistes en appelant récursivement la fonction *raytrace* depuis le point de réflexion, en regardant en direction du rayon réfléchi. C'est précisément ce qui est mis en oeuvre ci-dessous, où l'on prend garde à bien normaliser le nouveau vecteur $\vec{D}$ et à ne pas calculer un nombre infini de réflexions (cf. variable *bounce*, incrémentée à chaque appel à *raytrace*  et limitant le calcul à 3 réflexions consécutives).

L'intensité de la réflexion est ajustée à l'aide du coefficient _mirror_ caractéristique de chacune des surfaces, correspondant ici  à un attribut des sphères. 

In [31]:
class MirrorSphere(ShinySphere):
    
    def __init__(self, center, r, color, lambert = 1.0, specular = 0.5, phong = 70, mirror = 0.5):
        ShinySphere.__init__(self, center, r, color, lambert, specular, phong)
        self.mirror = mirror
    
    # Noter le nouveau paramètre bounce
    def light(self, O, D, d, scene, bounce):
        M = (O + D * d)                 # point sur l'objet
        N = self.normal(M)              # normale au point M
        toO = (scene.camera - M).norm() # vecteur depuis l'observateur vers le point M
        nudged = M + N * .0001          # point légèrement éloigné sur la normale en M pour éviter de se voir soi-même

        # Couleur locale au point M
        objcolor = self.diffusecolor(M)

        # Eclairage ambiant
        a = scene.ambient
        color = a + objcolor.mul(a)

        # boucle sur les sources de lumière
        for light in scene.sources:
            toL = (light.position - M).norm()    # direction sous laquelle M voit la source de lumière
            distL = abs(light.position - M)      # distance à laquelle M voit la source de lumière
            
            # M voit-il la source de lumière ?
            light_distances = [obj.intersect(nudged, toL, scene.FARAWAY) for obj in scene.objects]
            light_nearest = functools.reduce(np.minimum, light_distances)
            seelight = distL < light_nearest

            # Lambert (réflexion diffuse)
            lv = np.maximum(N.dot(toL), 0)
            c = objcolor.mul(light.color)
            color += c * self.lambert * lv * seelight

            # Blinn-Phong (réflexion spéculaire)
            phong = N.dot((toL + toO).norm())
            color += light.color * self.specular * np.power(np.clip(phong, 0, 1), self.phong) * seelight

        # prise en compte des réflexions (effet miroir)
        # Noter l'incrémentation du paramètre bounce pour l'appel à raytrace
        if bounce < scene.max_bounce:
            rayD = (D - N * 2 * D.dot(N)).norm()
            color += scene.raytrace(nudged, rayD, bounce + 1) * self.mirror
        
        return color

class MirrorCheckeredSphere(MirrorSphere):
    def diffusecolor(self, M):
        checker = (((M.x * 2 - 1000.5).astype(int) % 2) == ((M.z * 0.2 - 1000.25).astype(int) % 2))
        return rgb(
            np.where(checker,self.color[0].x,self.color[1].x),
            np.where(checker,self.color[0].y,self.color[1].y),
            np.where(checker,self.color[0].z,self.color[1].z)
        )

class MirrorScene(AmbientScene):
    
    def __init__(self, name, sources, objects, ambient=None, max_bounce=1, faraway=1.0e39):
        ambient = ambient if ambient else rgb(0,0,0)
        AmbientScene.__init__(self,name,sources,objects,ambient,faraway)
        self.max_bounce = max_bounce

    # Noter le nouveau paramètre bounce
    def raytrace(self, O, D, bounce):
        
        distances = [obj.intersect(O, D, self.FARAWAY) for obj in self.objects]
        nearest = functools.reduce(np.minimum, distances)
        color = rgb(0, 0, 0)

        for (s, d) in zip(self.objects, distances):
            color += s.light(O, D, d, scene, bounce) * (nearest != self.FARAWAY) * (d == nearest)
        return color
    
    # Noter l'appel de raytrace avec bounce = 0
    def trace (self,silent = False):
        t0 = time.perf_counter()
        self.data = self.raytrace(self.camera, (self.Q - self.camera).norm(), 0)
        self.ptime = time.perf_counter() - t0
        if ( not silent ):
            print("{} took {:.3f}s".format(self.name,self.ptime))
        return self

__Q17. Scène avec effet miroir :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Créer une scène comportant des objets réfléchissants.
<br><br>
__Attention :__ le temps de calcul croît très rapidement avec le nombre d'objets, de sources de lumière, et surtout le nombre de reflets successifs (paramètre <i>max_bounce</i>). Avec 3 sphères, deux sources et <i>max_bounce = 3</i> on arrive typiquement à une dizaine de secondes sur un PC raisonnable pour une image de 400x300 pixels
</div>

In [32]:
# votre code ici

<table style="border-color:transparent; border-spacing:50px 0; border-collapse:separate"><tr>
<td style="border:none"><img src="ref_mirror_scene.png" width="221"><br>
<center><i>exemple de résultat</i></center>
</td>
<td style="border:none"><img src="mirror_scene.png" width="221"><br>
<center><i>votre résultat</i></center>
</td>
</tr><tr>
<td style="border:none; text-align:center" colspan="2"><i>question 17. <a href="mirror_scene.png">mirror_scene.png</a></i></td>
</tr></table>

## 5. Optimisation du temps de calcul

Comme on a pu s'en apercevoir, dès que l'on prend en compte les réflexions, le temps de calcul commence à devenir significatif. Il est néanmoins possible de gagner beaucoup de temps. 

En effet, _raytrace()_ appelle la méthode _light()_ pour tous les pixels, qu'ils soient situés à l'infini ou sur un objet, que ces derniers soient vu de l'observateur ou cachés. Rappelez-vous la question 7, la sphère ne représentait que 13 898 pixels sur les 120 000 de l'image : il y a certainement du temps à gagner ici !

La version ci-dessous extrait les points situés sur un objet et vus par l'observateur, calcule la couleur du pixel uniquement pour ces points-là, puis les réinjecte dans un vecteur ayant le nombre initial d'éléments.

In [33]:
class OptimizedScene(MirrorScene):
 
    def raytrace(self, O, D, bounce):
        
        distances = [obj.intersect(O, D, self.FARAWAY) for obj in self.objects]
        nearest = functools.reduce(np.minimum, distances)
        color = rgb(0, 0, 0)

        for (obj, dist) in zip(self.objects, distances):

            # points non situés à l'infini, et non cachés par un autre objet
            hit = (nearest != self.FARAWAY) & (dist == nearest)
            if np.any(hit):

                # on extrait les points vus
                dc = vec3.extract_component(None, hit, dist)
                Oc = O.extract(hit)
                Dc = D.extract(hit)
            
                # on calcule la couleur des points vus
                cc = obj.light(Oc, Dc, dc, self, bounce)
            
                # on replace les points calculés dans l'ensemble des points 
                color += cc.place(hit)
                
        return color

Il faut également redéfinir les classes _vec3_ et _rgb_ pour ajouter les méthodes _extract()_ et _place()_
utilisées par la nouvelle implémentation de _raytrace()_

In [34]:
class vec3():
    def __init__(self, x, y, z):
        (self.x, self.y, self.z) = (x, y, z)

    # multiplication par une constante : v * k (attention k * v génère une erreur)
    def __mul__(self, other):
        return vec3(self.x * other, self.y * other, self.z * other)

    # addition de deux vecteurs : v1 + v2
    def __add__(self, other):
        return vec3(self.x + other.x, self.y + other.y, self.z + other.z)

    # soustraction de deux vecteurs : v1 - v2
    def __sub__(self, other):
        return vec3(self.x - other.x, self.y - other.y, self.z - other.z)

    # produit scalaire : v1.dot(v2) = v1.v2
    def dot(self, other):
        return (self.x * other.x) + (self.y * other.y) + (self.z * other.z)

    # Carré de la valeur absolue : abs2(v) = v.v
    def abs2(self):
        return self.dot(self)
    
    # valeur absolue : abs(v) = sqrt(v.v)
    def __abs__(self):
        return np.sqrt(self.dot(self))

    # normalisation : v = v.norm() * sqrt(v.v)
    def norm(self):
        mag = abs(self)
        return self * (1.0 / np.where(mag == 0, 1, mag))

    # composantes : v.components() = (x,y,z)
    def components(self):
        return (self.x, self.y, self.z)

    # affichage : print(v)
    def __str__(self):
        f = "{}({:.3f}, {:.3f}, {:.3f})" if isinstance(self.x, numbers.Number) else "{}({}, {}, {})"
        return f.format(self.__class__.__name__,*self.components())
    
    def extract(self, cond):
        return self.__class__(
            self.extract_component(cond, self.x),
            self.extract_component(cond, self.y),
            self.extract_component(cond, self.z))

    def extract_component(self,cond, x):
        if isinstance(x, numbers.Number):
            return x
        else:
            return np.extract(cond, x)

    def place(self, cond):
        r = self.__class__(np.zeros(cond.shape), np.zeros(cond.shape), np.zeros(cond.shape))
        np.place(r.x, cond, self.x)
        np.place(r.y, cond, self.y)
        np.place(r.z, cond, self.z)
        return r

class rgb(vec3):
    def mul(self, other):
        return rgb(self.x * other.x, self.y * other.y, self.z * other.z)

abs2 = vec3.abs2

__Q18. Calcul de scène optimisé :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Retracer la scène précédente avec le calcul optimisé et comparer les temps de calcul.
</div>

In [35]:
# votre code ici

<table style="border-color:transparent; border-spacing:50px 0; border-collapse:separate"><tr>
<td style="border:none"><img src="non_optimized_scene.png" width="221"><br>
<center><i><a href="non_optimized_scene.png">non_optimized_scene.png</a></i></center>
</td>
<td style="border:none"><img src="optimized_scene.png" width="221"><br>
<center><i><a href="optimized_scene.png">optimized_scene.png</a></i></center>
</td>
</tr><tr>
<td style="border:none; text-align:center" colspan="2"><i>question 18. images identiques pour temps de calcul divisé par 30</i></td>
</tr></table>

On gagne un facteur entre 20 et 100 ! La raison pour laquelle cette optimisation est si efficace est que la méthode _raytrace()_ est non seulement appelée pour les rayons primaires, mais également pour le calcul des réflexions.

## 6. Pour ceux qui veulent aller plus loin

Pour la réalisation de votre projet, un module nommé _raytracer_ vous sera fourni. En attendant, le module <i>simple_raytracer</i> dont le code source est joint aux documents que vous avez reçus pour ce TD, implémente les classes _vec3, rgb, Scene, Sphere et CheckeredSphere_ avec essentiellement le code dont vous avez pu découvrir la justification ci-dessus.

Petites différences tout de même :<br>
&bull; la couleur d'un objet est un paramètre nommé _diffuse,_<br>
&bull; les objets disposent d'un coefficient de sensibilité à la lumière ambiante nommé _ambient,_<br>
&bull; comme _diffuse,_ les "coefficients" _ambient, specular,_ et _mirror_ peuvent s'exprimer sous forme d'un triplet _rgb,_<br>
&bull; on peut modifier la fonction de sélection de la couleur des objets bicolores comme _CheckeredSphere._<br>

Cette dernière possibilité est illustrée dans l'extrait:

Cerise sur le gâteau, ce module implémente d'autres types d'objets comme _Plane, CheckeredPlane, Polygon, Disk, Pipe, PipeSection, Cylinder, Cube, Cone, ConeTrunk, BasedConeTrunk_ dont nous vous laissons le loisir de découvrir à la fois le code source et les modalités d'usage. Les principes du calcul de l'intersection d'un volume avec une droite &ndash; méthode _intersect()_ &ndash; et du calcul de la normale en un point &ndash; méthode _normal()_ &ndash; sont détaillées dans le notebook <a href="Mod%C3%A9lisation%20des%20artefacts.ipynb">_"Modélisation des artefacts"_</a> joint à ce TD.

__Q19. Utilisation du module <i>simple_raytracer</i> :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Créer des scènes en faisant appel au module <i>simple_raytracer</i> :
<br><br>
<pre style="margin: 0; background-color:#eef">from simple_raytracer import *</pre>
</div>

In [36]:
# votre code ici

<table style="border-color:transparent; border-spacing:50px 0; border-collapse:separate"><tr>
<td style="border:none"><img src="ref_raytracer_scene.png" width="221"><br>
<center><i>exemple de résultat</i></center>
</td>
<td style="border:none"><img src="raytracer_scene.png" width="221"><br>
<center><i>votre résultat</i></center>
</td>
</tr><tr>
<td style="border:none; text-align:center" colspan="2"><i>question 19. <a href="raytracer_scene.png">raytracer_scene.png</a></i></td>
</tr></table>

## Références

Ce sujet de TD, les exemples de code qu'il comporte, ainsi que le module _raytracer_ mis à disposition pour le développement du projet ont été conçus sur la base de l'article :

- _"A reasonably speedy Python ray-tracer",_ James Bowman, [<a href="http://www.excamera.com/sphinx/article-ray.html">en ligne</a>], consulté le 12/09/2017

Pour une implémentation simplifiée on pourra consulter le code de Cyrille Rossant, sur lequel le précédent est lui-même basé :

- _"Very simple ray tracing engine in (almost) pure Python",_ Cyrille Rossant, [<a href="https://gist.github.com/rossant/6046463">en ligne</a>], 
consulté le 12/09/2017

Une autre référence très utile avec des explications très détaillées, et des exemples en pseudo-code ou en C++ :

- _"Introduction to Ray Tracing: a Simple Method for Creating 3D Images",_ Scratchapixel,
[<a href="https://www.scratchapixel.com/lessons/3d-basic-rendering/introduction-to-ray-tracing">en ligne</a>], consulté le 12/09/2017

Et un cours sur les principes du lancer de rayons :

- _"CS4620/5620: Lecture 8 - Ray Tracing Basics",_ Kavita Bala, Cornell University,
[<a href="http://www.cs.cornell.edu/courses/cs4620/2011fa/lectures/08raytracingWeb.pdf">en ligne</a>],
consulté le 12/09/2017